# Final Project Submission

- Student Name: Deztany Jackson
- Student Pace: Self Pace
- Scheduled Project Review: April 7, 2023
- Instructor Name: Morgan Jones



Main Library Elemenets:
[Checklist](https://docs.google.com/document/d/1swj3BthL_M1IzJzkOmzhCWc595sRC6WrFI8S-xTKCr0/edit)

## Business Understanding

# Clean up later
- Real estate company (Primary Stakeholder) wanting to partner with the local pet adoption agency
    - Adoption, Animals, Buyers/Sellers (potential adoptees), Adoptees (potenial customers), Data Scientist
    - Wants to minimize danger and make neighbor hoods attractive while caring for animals and potentially getting customers to to think about housing needs and pet adoption at same time
Assumptions:
    - The RE is not a SME in adoption and animals, so they only care if an animal wiill be adopted by closing - Binary Classification
    [https://towardsdatascience.com/the-3-most-important-basic-classification-metrics-3368dd425f74](https://towardsdatascience.com/the-3-most-important-basic-classification-metrics-3368dd425f74)
    - Want Precision to maximize TP and minimize FP of adoptees when I have results that tell me a lot of animals will be adopted.  Dont want to sell or rent and put integrity on line...
    - RE agents care about knowing the true total when the TP is low and a higher FN.. ...Recall
Questions: 
    - What is the average time until close
    - What happens with most animals when people are evicted?
    - Why are most animals given up for adoption?
    - What area are we focused on?
Metrics:
    AUC-ROC True Positive Rate against False Positive Rate
    Minimize False Positives (Precision)

## Import Libraries

In [16]:
import os
import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#Import Knn

from sklearn.metrics import ConfusionMatrixDisplay,classification_report,recall_score,accuracy_score, auc
from sklearn.metrics import precision_score, f1_score, confusion_matrix,  RocCurveDisplay
from sklearn.metrics import precision_recall_curve,average_precision_score, make_scorer
from sklearn.metrics import PrecisionRecallDisplay, PredictionErrorDisplay
from sklearn import tree

# Import tensorflow stuff from Model 4

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

## Data Import

In [17]:
cur_dir = os.getcwd()
cur_dir

'/Users/deztanyjackson/Documents/0_Flatiron/Capstone_P5'

In [49]:
# Import training csv file
train_df = pd.read_csv("data/train/train.csv")
test_df = pd.read_csv("data/test/test.csv")
state_df = pd.read_csv("data/StateLabels.csv")
color_df = pd.read_csv("data/ColorLabels.csv")
breed_df = pd.read_csv("data/BreedLabels.csv")

This reference is wrong find right one...was loan predicitons
[Loan Classification,Kamal Khumar, 2021 ](https://www.kaggle.com/code/kamalkhumar/loan-status-prediction)

## Data Understanding

### Dataset Description

In [55]:
# Come back to the Categorical Values of the AdoptionSpeed

<table style="width:100%">
<thead>
<tr>
<th style="text-align:center; font-weight: bold; font-size:14px">Feature </th>
<th style="text-align:center; font-weight: bold; font-size:14px">Description</th>
<th style="text-align:center; font-weight: bold; font-size:14px">Data Type</th> 
</tr>
</thead>
<tbody>
<td><b>Type</b></td>
<td>Type of Animal<br> (1=Dog,2=Cat)</td>
<td>Nominal Number</td>    
<tr> 
<td><b>Name</b></td>
<td>Name of Pet<br> (Empty if not named)</td>
<td>String</td>
</tr>
<tr> 
<td><b>Age</b></td>
<td>Age of pet when listed<br> (Months)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Breed</b></td>
<td>Breed1 (Primary breed),Breed2 (Secondary breed, if mixed) <br> (Ref Breedlabels dict)</td>
<td>Nominal Number:String </td>
</tr>
<tr>
<td><b>Gender</b></td>
<td>Gender of pet <br> 1=Male, 2=Female, 3=Mixed, if profile represents group of pets)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Color</b></td>
<td>Colors of pets<br>Ref Colorlabels dict</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Maturity Size</b></td>
<td>Number of Rooms <br>(1=Sm, 2=Med,3=Lg,4=XLg,0=Not Spec)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Fur Length</b></td>
<td>Fur Length<br>(1=Shrt, 2=Med,3=Lg,4=Lng,0=Not Spec) </td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Vaccinated</b></td>
<td>Pet has been vaccinated<br>(1=Yes,2=No,3=Not Sure)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Dewormed</b></td>
<td>Pet has been dewormed<br>(1=Yes,2=No,3=Not Sure)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Sterilized</b></td>
<td>Pet has been spayed/neutered<br>(1=Yes,2=No,3=Not Sure)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Health</b></td>
<td>Health Condition<br>(1=Healthy,2=Minor Inj,3=Serious Inj,0=Not Spec)</td>
<td>Nominal</td>
</tr>
<tr>
<td><b>Quantity</b></td>
<td>Number of pets represented in profile<br></td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Fee</b></td>
<td>Adoption fee<br> (0=Free)</td>
<td>Cardinal Number</td>
</tr>
<tr>
<td><b>state</b></td>
<td>State location in Malaysia <br>(Ref to StateLabels dict)</td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>RescuerID</b></td>
<td>Unique hash ID of rescuer <br></td>
<td>String</td>
</tr>
<tr>
<td><b>VideoAmt</b></td>
<td>Total uploaded videos for pet <br></td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>PhotoAmt</b></td>
<td>Total uploaded videos for pet <br></td>
<td>Nominal Number</td>
</tr>
<tr>
<td><b>Description</b></td>
<td>Profile write-up <br>(Primarily Eng, also Malay or Chinese)</td>
<td>String Object</td>  
</tr>
<tr>    
<td><b>AdoptionSpreed</b></td>
<td>Categorical speed of adoption. Lower is faster.<br></td>
<td>Nominal Number</td>  
</tr>
<tr>    
<td><b>PetID</b></td>
<td>Unique hash ID of pet profile<br></td>
<td>String Object</td> 
</tbody>
</table>

### Exploring Dataset

The dataset from Kaggle had several files that will be used in the modeling effort.<br>
- Training set (14993 entries)
- Test set (3972 entries) 
- Breed labels (307 entries)
- Color labels ( 7 entries)
- State labels (15 entries)

EDA CHeatsheet https://datasans.medium.com/exploratory-data-analysis-cheatsheet-everything-you-might-need-36230538484d
Review the values for the labels
Review missing values
review odd inputs for values (naan,)
review columns that are not needed
look at any imbalances
correlation map
    Do any need to be removed
histogram
check for outliers with scattering

bar graph of primary correlated values
use describe...
What insights can you get from the percentages and plots
Does anything need to be standardized?  Do we standardize categorical numbers?
What plots will we use


Feature Engineering
https://www.projectpro.io/article/8-feature-engineering-techniques-for-machine-learning/423Bin the speed categories for ease of the business problem

In [ ]:
# Exploring training set

In [56]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [2]:
#Describe the data sources and explain why the data are suitable for the project
#Present the size of the dataset and descriptive statistics for all features used in the analysis
#Justify the inclusion of features based on their properties and relevance for the project
#Identify any limitations of the data that have implications for the project


In [ ]:
#Import labels as dictionaries, use  at end for presenting and specifically understand checking names
#

In [28]:
#If profile represents multiple bets, we will separate out or delete

## Data Preparation

## Modeling

In [ ]:
### 

### Final Model

## Evaluation

### Conclusion

### Reccomendations

### Limitations